# Lab 3 — clustering

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle

%matplotlib inline
plt.style.use("ggplot")

In [ ]:
with open("tagsCoordinates.pickle", "rb") as f:
    tagsCoord = pickle.load(f, encoding="utf-8")

In [ ]:
from sklearn.cluster import KMeans

twoDir = np.zeros([len(tagsCoord),2])
threeDir = np.zeros([len(tagsCoord),3])
fourDir = np.zeros([len(tagsCoord),4])
fiveDir = np.zeros([len(tagsCoord),5])

i = 0
for tags,coord in tagsCoord.items():
    twoDir[i,0] = coord[0]
    twoDir[i,1] = coord[1]
    i += 1

kmeans = KMeans(n_clusters=2).fit(twoDir)
print(kmeans.cluster_centers_ )

In [ ]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, ResetTool, PanTool, WheelZoomTool, SaveTool
output_notebook()

In [ ]:
# Categorial colors
from bokeh.palettes import Dark2_8

source = ColumnDataSource(
    data={
        "x": [x[0] for x in data],
        "y": [x[1] for x in data],
        "name": [x[2] for x in data],
        "color": [Dark2_8[x[4]] for x in data],
    })

hover = HoverTool(
    tooltips=[
        ("Name", "@name"),
    ])
tools = [hover, ResetTool(), PanTool(), WheelZoomTool(), SaveTool()]

p = figure(plot_width=960, plot_height=360, tools=tools, title="Mouse over the dots")
p.circle("x", "y", source=source, size=20, color="color", alpha=0.5)
show(p, notebook_handle=True)